In [2]:
from data_fetching import fetch_solutions_single, fetch_solutions_batch
from mechanism import (
    Trade,
    Solution,
    FilterRankRewardMechanism,
    VCGRewardMechanism,
    NoFilter,
    BaselineFilter,
    TokenPairs,
    TradedTokens,
    SingleWinner,
    TokenPairFilteringWinners,
    SubsetFilteringWinners,
    BatchSecondPriceReward,
    BatchOverlapSecondPriceReward,
    TokenPairImprovementReward,
)

In [11]:
def compute_reward_statistic(solutions_batch, rewards_batch):
    print(f"number of auction: {len(solutions_batch)}\n")
    print(
        "average number of winners per auction:\n"
        f"single winner: {sum(len(rewards[0]) for rewards in rewards_batch) / len(rewards_batch)}\n"
        f"multiple winners: {sum(len(rewards[1]) for rewards in rewards_batch) / len(rewards_batch)}\n"
        f"vcg winners: {sum(len(rewards[2]) for rewards in rewards_batch) / len(rewards_batch)}\n"
    )
    solutions_batch_dicts = [
        {solution.id: solution for solution in solutions}
        for solutions in solutions_batch
    ]
    print(
        "average total score per auction:\n"
        f"single winner: {sum(sum(solutions_batch_dicts[i][solution_id].score for solution_id in rewards[0]) for i, rewards in enumerate(rewards_batch)) / len(rewards_batch)}\n"
        f"multiple winners: {sum(sum(solutions_batch_dicts[i][solution_id].score for solution_id in rewards[1]) for i, rewards in enumerate(rewards_batch)) / len(rewards_batch)}\n"
        f"vcg winners: {sum(sum(solutions_batch_dicts[i][solution_id].score for solution_id in rewards[2]) for i, rewards in enumerate(rewards_batch)) / len(rewards_batch)}\n"
    )
    print(
        "average rewards per winner:\n"
        f"single winner: {sum(sum(reward for reward, _ in rewards[0].values()) for rewards in rewards_batch) / sum(len(rewards[0]) for rewards in rewards_batch) / 10**18}\n"
        f"multiple winners: {sum(sum(reward for reward, _ in rewards[1].values()) for rewards in rewards_batch) / sum(len(rewards[1]) for rewards in rewards_batch) / 10**18}\n"
        f"vcg winners: {sum(sum(reward for reward, _ in rewards[2].values()) for rewards in rewards_batch) / sum(len(rewards[2]) for rewards in rewards_batch) / 10**18}\n"
    )
    print(
        "average rewards/penalties:\n"
        f"single winner: {sum(sum(penalty for _, penalty in rewards[0].values()) for rewards in rewards_batch) / sum(len(rewards[0]) for rewards in rewards_batch) / 10**18}\n"
        f"multiple winners: {sum(sum(penalty for _, penalty in rewards[1].values()) for rewards in rewards_batch) / sum(len(rewards[1]) for rewards in rewards_batch) / 10**18}\n"
        f"vcg winners: {sum(sum(penalty for _, penalty in rewards[2].values()) for rewards in rewards_batch) / sum(len(rewards[2]) for rewards in rewards_batch) / 10**18}\n"
    )

In [3]:
solutions_batch = [
    [  # batch vs single order solutions
        Solution(
            "batch winner",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 100), Trade("2", "C", "D", 100)],
        ),
        Solution(
            "best on first trade",
            solver="solver 2",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "best on second trade",
            solver="solver 3",
            score=150,
            trades=[Trade("2", "C", "D", 150)],
        ),
    ],
    [  # solutions without overlap
        Solution(
            "best on first trade",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=140,
            trades=[Trade("2", "C", "D", 140)],
        ),
        Solution(
            "bad batch",
            solver="solver 3",
            score=100,
            trades=[Trade("1", "A", "B", 50), Trade("2", "C", "D", 50)],
        ),
    ],
    [  # batch in between solutions without overlap
        Solution(
            "best on first trade",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 150)],
        ),
        Solution(
            "batch with overlap",
            solver="solver 3",
            score=100,
            trades=[Trade("1", "A", "B", 50), Trade("2", "C", "D", 50)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=90,
            trades=[Trade("2", "C", "D", 90)],
        ),
    ],
    [  # reference is not from winner
        Solution(
            "batch with overlap",
            solver="solver 1",
            score=200,
            trades=[Trade("1", "A", "B", 150), Trade("2", "C", "D", 50)],
        ),
        Solution(
            "best on first trade",
            solver="solver 1",
            score=100,
            trades=[Trade("1", "A", "B", 100)],
        ),
        Solution(
            "best on second trade",
            solver="solver 2",
            score=90,
            trades=[Trade("2", "C", "D", 90)],
        ),
    ],
    [  # token overlap but not on the same token pair
        Solution(
            "batch with overlap",
            solver="solver 1",
            score=100,
            trades=[Trade("1", "A", "B", 100)],
        ),
        Solution(
            "best on first trade",
            solver="solver 2",
            score=90,
            trades=[Trade("1", "A", "C", 90)],
        ),
    ],
    [
        Solution(
            id="batch winner",
            solver="solver 1",
            score=150,
            trades=[Trade("1", "A", "B", 100), Trade("2", "A", "C", 50)],
        ),
        Solution(
            id="unfair batch",
            solver="solver 2",
            score=110,
            trades=[Trade("1", "A", "B", 50), Trade("2", "A", "C", 60)],
        ),
        Solution(
            id="overlapping batch",
            solver="solver 3",
            score=100,
            trades=[Trade("3", "B", "A", 50), Trade("2", "A", "C", 50)],
        ),
        Solution(
            id="non-overlapping batch",
            solver="solver 4",
            score=100,
            trades=[Trade("3", "B", "A", 40), Trade("4", "D", "E", 60)],
        ),
        Solution(
            id="non-overlapping batch unfair",
            solver="solver 5",
            score=120,
            trades=[Trade("3", "B", "A", 20), Trade("4", "D", "E", 100)],
        ),
        Solution(
            id="reference A->B",
            solver="solver 1",
            score=80,
            trades=[Trade("1", "A", "B", 80)],
        ),
        Solution(
            id="reference A->C",
            solver="solver 2",
            score=40,
            trades=[Trade("2", "A", "C", 40)],
        ),
        Solution(
            id="runner up A->B",
            solver="solver 2",
            score=40,
            trades=[Trade("1", "A", "B", 40)],
        ),
        Solution(
            id="runner up A->C",
            solver="solver 1",
            score=40,
            trades=[Trade("2", "A", "C", 40)],
        ),
        Solution(
            id="reference B->A",
            solver="solver 7",
            score=30,
            trades=[Trade("3", "B", "A", 30)],
        ),
        Solution(
            id="reference F->G",
            solver="solver 8",
            score=50,
            trades=[Trade("5", "F", "G", 50)],
        ),
        Solution(
            id="runner up F->G",
            solver="solver 1",
            score=40,
            trades=[Trade("5", "F", "G", 40)],
        ),
        Solution(
            id="reference H->I",
            solver="solver 8",
            score=50,
            trades=[Trade("6", "H", "I", 50)],
        ),
    ],
]

In [7]:
solutions_batch = fetch_solutions_batch(9499893, 9532691)

2024-10-24 17:38:36,822 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-10-24 17:38:36,823 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-10-24 17:38:36,852 INFO sqlalchemy.engine.Engine select current_schema()
2024-10-24 17:38:36,853 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-10-24 17:38:36,891 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-10-24 17:38:36,892 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-10-24 17:38:36,976 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-24 17:38:36,977 INFO sqlalchemy.engine.Engine select * from solver_competitions where id between %(start_id)s and %(end_id)s
2024-10-24 17:38:36,977 INFO sqlalchemy.engine.Engine [generated in 0.00114s] {'start_id': 9499893, 'end_id': 9532691}
2024-10-24 17:42:33,224 INFO sqlalchemy.engine.Engine ROLLBACK
2024-10-24 17:42:43,883 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-10-24 17:42:43,884 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-10-24 17:42:

In [12]:
mechanisms = [
    FilterRankRewardMechanism(
        NoFilter(),
        SingleWinner(),
        BatchSecondPriceReward(12 * 10**15, 10**16),
    ),
    FilterRankRewardMechanism(
        NoFilter(),
        SubsetFilteringWinners(
            filtering_function=TradedTokens(), cumulative_filtering=True
        ),
        BatchOverlapSecondPriceReward(12 * 10**15, 10**16, TradedTokens()),
    ),
    VCGRewardMechanism(
        NoFilter(),
        SubsetFilteringWinners(
            filtering_function=TradedTokens(), cumulative_filtering=True
        ),
    ),
]
all_rewards: list[list[dict[str, tuple[int, int]]]] = []
for solutions in solutions_batch:
    rewards = [mechanism.winners_and_rewards(solutions) for mechanism in mechanisms]

    all_rewards.append(rewards)

compute_reward_statistic(solutions_batch, all_rewards)

number of auction: 17983

average number of winners per auction:
single winner: 1.0
multiple winners: 1.0499360507145639
vcg winners: 1.0568870600011122

average total score per auction:
single winner: 9.083107006308997e+16
multiple winners: 9.1068465712421e+16
vcg winners: 9.110020430494392e+16

average rewards per winner:
single winner: 0.0020486283540665767
multiple winners: 0.0020649814762660956
vcg winners: 0.005962138506552275

average rewards/penalties:
single winner: -0.006130556414858743
multiple winners: -0.005833850437095657
vcg winners: -0.08023458747554846

